In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn

def load_data():
    print("📥 Loading churn dataset...")
    df = pd.read_csv(r'D:\Github_Share\Dataset\WA_Fn-UseC_-Telco-Customer-Churn.csv')

    print("🧹 Cleaning and preprocessing data...")
    df = df[df["TotalCharges"].str.strip() != ""]
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"])
    df.drop(["customerID"], axis=1, inplace=True)
    df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})
    df = pd.get_dummies(df)

    selected_features = [
        "tenure",
        "MonthlyCharges",
        "TotalCharges",
        "Contract_Month-to-month",
        "InternetService_Fiber optic",
        "TechSupport_No",
        "OnlineSecurity_No",
        "PaymentMethod_Electronic check"
    ]

    for col in selected_features:
        if col not in df.columns:
            df[col] = 0

    X = df[selected_features]
    y = df["Churn"]

    print("✅ Features prepared. Proceeding to split...")
    return train_test_split(X, y, test_size=0.2, random_state=42)

def train_and_log():
    X_train, X_test, y_train, y_test = load_data()

    print("🚀 Training Logistic Regression model...")
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    print("📊 Evaluating model...")
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"🎯 Accuracy: {acc:.4f}")

    print("📝 Logging model and metrics to MLflow...")
    mlflow.set_experiment("Churn-Prediction-LogisticRegression")
    input_example = X_test.iloc[:1]

    with mlflow.start_run():
        mlflow.sklearn.log_model(model, "model", input_example=input_example)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_params(model.get_params())
        print("✅ Model logged to MLflow successfully.")

if __name__ == "__main__":
    train_and_log()

📥 Loading churn dataset...
🧹 Cleaning and preprocessing data...
✅ Features prepared. Proceeding to split...
🚀 Training XGBoost model...
📊 Evaluating model...
🎯 Accuracy: 0.7790
📝 Logging model and metrics to MLflow...


d:\pyenv\pyenv-win\versions\3.12.3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
d:\pyenv\pyenv-win\versions\3.12.3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missi

✅ Model logged to MLflow successfully.
